## Star-Galaxy separation using Decision Trees and Random Forests 

Background: One important task in current and future astronomical surveys is to separate the surveyed objects into (mainly) stars and galaxies and also to a special category of very active galaxies known as quasars.

Usually in these large astronomical surveys we are interested in having a pure sample of galaxies (thus stars act more like a nuisance) to use for cosmological analyses. In some cases we are also interested in a quasar catalog for studies of that particular galaxy population.

In this notebook we use data from the 17th data release (DR 17) of the Sloan Digital Sky Survey SDSS to classify objects into the three categories (QSO, Star, Galaxy) based on their photometric properties (flux, that is light intensities, in the filters 
$u$, $g$, $r$, $i$, $z$) and derivative quantities.

Although you can easily query the SDSS from the survey's website, we used a dataset hosted at Kaggle: https://www.kaggle.com/datasets/fedesoriano/stellar-classification-dataset-sdss17

# Just run the cells below

You can simply run the cells in this section - they are mostly about reading and preparing the data. You can read more about them at home.

In [1]:
# Import basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Import and Prepare data


In [4]:
# Let's read the data
SDSS_table = pd.read_csv("Star_Galaxy_Quasar.csv")
SDSS_table.head()

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842


### Features (predictors) 

For the meaning of the name of the columns, you can refer to the link above. 

For now, we will consider as **features** only the magnitudes $u$, $g$, $r$, $i$, $z$
and as labels that we want to predict the `class` column

In [23]:
# Matrix of features
X_feat = SDSS_table.loc[:,'u':'z']
X_feat.head()

,u,g,r,i,z
0,23.87882,22.27530,20.39501,19.16573,18.79371
1,24.77759,22.83188,22.58444,21.16812,21.61427
2,25.26307,22.66389,20.60976,19.34857,18.94827
3,22.13682,23.77656,21.61162,20.50454,19.25010
4,19.43718,17.58028,16.49747,15.97711,15.54461


However if we keep only the magnitudes, the predicive power will be low. We can create some combinations of data, such as colors (differences in the filters) 
and ratios of the filters

In [43]:
# Let's download the magnitudes here again, separately
u = SDSS_table['u'];g = SDSS_table['g'];r = SDSS_table['r'];i = SDSS_table['i'];z =SDSS_table['z']

# Define colors 
u_g = u-g; u_r = u-r; u_i = u-i; u_z = u-z; g_r = g-r; g_i = g-i; g_z = g-z; r_i = r-i; r_z = r-z

# Define the ratios now
u_ov_g = u/g; u_ov_r = u/r; u_ov_i = u/i; u_ov_z = u/z; g_ov_r = g/r; g_ov_i = g/i; g_ov_z=g/z; r_ov_i = r/i
r_ov_z = r/z 


# Create again a feature matrix with all the features
n_feat = len(u)

# Initialize feature matrix
X_feat = np.zeros([n_feat,23])
# ================================================================
# ================================================================
# Define feature matrix
X_feat[:,0] = u;X_feat[:,1] = g; X_feat[:,2] = r; X_feat[:,3] = i; X_feat[:,4] =z
X_feat[:,5] = u_g;X_feat[:,6] = u_r;X_feat[:,7] = u_i;X_feat[:,8] = u_z; X_feat[:,9] = g_r
X_feat[:,10] = g_i; X_feat[:,11] = g_z; X_feat[:,12] = r_i; X_feat[:,13] = r_z
X_feat[:,14] = u_ov_g; X_feat[:,15] = u_ov_r; X_feat[:,16] = u_ov_i; X_feat[:,17] = u_ov_z
X_feat[:,18] = g_ov_r;X_feat[:,19] = g_ov_i;X_feat[:,20] = g_ov_z; X_feat[:,21] = r_ov_i; X_feat[:,22] = r_ov_z

### Target Labels

In [37]:
# Let's also get the labels
y_label = SDSS_table['class'].values

print(y_label[0:10])

print(len(y_label))

print('Number of Galaxies:',len(y_label[y_label=='GALAXY']))
print('Number of Stars:',len(y_label[y_label=='STAR']))
print('Number of QSOs:',len(y_label[y_label=='QSO']))

['GALAXY' 'GALAXY' 'GALAXY' 'GALAXY' 'GALAXY' 'QSO' 'QSO' 'GALAXY'
 'GALAXY' 'STAR']
100000
Number of Galaxies: 59445
Number of Stars: 21594
Number of QSOs: 18961


Now let us change the class of objects to something that a machine can understand. Say "GALAXY" = 0, "STAR" = 1, "QSO"=2

In [32]:
y_label_num = np.zeros(len(y_label))
for i_l in range(len(y_label)):
    
    if (y_label[i_l]=="GALAXY"):
        y_label_num[i_l] = 0
    elif (y_label[i_l]=="STAR"):
        y_label_num[i_l] = 1
    else:
        y_label_num[i_l] = 2

For the ease of the rest of the exercise, we omit the QSO class

In [45]:
# Star galaxy feature matrix
X_feat_SG = X_feat[(y_label_num==0.0)|(y_label_num==1.0),:]
# Star galaxy labels
y_label_SG = y_label_num[(y_label_num==0.0)|(y_label_num==1.0)]

# Exercises start here

Let's split the sample into training and test sets. We use `scikit-learn` for that.

In [49]:
print('Number of galaxies:',len(y_label_SG[y_label_SG==1]))
print('Number of Stars:',len(y_label_SG[y_label_SG==0]))

Number of galaxies: 21594
Number of Stars: 59445


In [46]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_feat_SG, y_label_SG, train_size = 0.70, random_state = 42)

### Problem 1: Decision Tree classification

Train Decision Tree classifier to separate stars from galaxies in your data.
Report the accuracy, precision, and recall both on the training and the test set. Do you see any signs of overfitting?

Use also the balanced accuracy, since the number of galaxies in our sample is smaller than 

**Hints**: Use the Desicion Tree classifier from `scikit-learn` here: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html,
with the default hyperparameters (e.g. no need to change anything at this point).

Sklearn also implements the metrics we are going to need, e.g. the accuracy here: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score. Be careful to have the correct order of predicted/true values.

### Problem 1 solution:

In [50]:
from sklearn.tree import DecisionTreeClassifier

# Define the decision tree
DT_clf = DecisionTreeClassifier()

# Fit
DT_clf.fit(X_train,y_train)

DecisionTreeClassifier()

Predict on the training and test sets

In [51]:
y_train_pred = DT_clf.predict(X_train) 
y_test_pred = DT_clf.predict(X_test)

Import all the metrics

In [53]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

Performance on the **training set**:

In [56]:
print('Accuracy:',accuracy_score(y_train,y_train_pred))
print('Balanced:',balanced_accuracy_score(y_train,y_train_pred))
print('Precision:',precision_score(y_train,y_train_pred))
print('Recall:',recall_score(y_train,y_train_pred))

Accuracy: 1.0
Balanced: 1.0
Precision: 1.0
Recall: 1.0


Performance on the **test set**:

In [58]:
print('Accuracy:',accuracy_score(y_test,y_test_pred))
print('Balanced:',balanced_accuracy_score(y_test,y_test_pred))
print('Precision:',precision_score(y_test,y_test_pred))
print('Recall:',recall_score(y_test,y_test_pred))

Accuracy: 0.9063836788417242
Balanced: 0.881908690138608
Precision: 0.8196018376722818
Recall: 0.8297674418604651


The results in both cases are good, but we can clearly see there is overfitting.

### Problem 2: Random Forest classification

Decision Trees are rarely used as stand-alone classifiers; instead they are used to construct the more powerful (and less prone to overfitting) random forests.

Train the default version of the Random Forest classifier from `scikit-learn` to predict whether a datapoint in your set is a star or a galaxy. Report the same metrics as before.

This may take a couple of minutes.

### Problem 2 Solution:

In [59]:
from sklearn.ensemble import RandomForestClassifier

# Define the Random Forest
RF_clf = RandomForestClassifier()

# Fit
RF_clf.fit(X_train,y_train)

RandomForestClassifier()

Predict on training and test sets

In [60]:
y_train_pred = RF_clf.predict(X_train) 
y_test_pred = RF_clf.predict(X_test)

Performance on the **training set**:

In [61]:
print('Accuracy:',accuracy_score(y_train,y_train_pred))
print('Balanced:',balanced_accuracy_score(y_train,y_train_pred))
print('Precision:',precision_score(y_train,y_train_pred))
print('Recall:',recall_score(y_train,y_train_pred))

Accuracy: 1.0
Balanced: 1.0
Precision: 1.0
Recall: 1.0


Performance on the **test set**:

In [62]:
print('Accuracy:',accuracy_score(y_test,y_test_pred))
print('Balanced:',balanced_accuracy_score(y_test,y_test_pred))
print('Precision:',precision_score(y_test,y_test_pred))
print('Recall:',recall_score(y_test,y_test_pred))

Accuracy: 0.939823955248437
Balanced: 0.9118974411053217
Precision: 0.9149608919953404
Recall: 0.8524031007751938


### Problem 3: Hyperparameter selection with the Cross-Validation score

We saw above that the decision tree overfits. One of the most important hyperparametrs we can tweak to prevent this is the maximum depth of the tree - the number of splits. By default, `scikit-learn` splits the dataset in such a way that each node is pure (has data points belonging to the same class) or just two data points.

Using the Cross-Validation method (you can select $k=3$ or $k=5$), select the optimal maximum depth of the tree.
You can use `scikit-learn` to calculate the cross validation score for a number of different values of the `max_depth` parameter. What is the optimal value?

What is the balanced accuracy in the training and test sets when the optimal value is used? Comment on overfitting.


**Hints:** Check the cross-validation score implementation here: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

Also, you will need to run a for loop, and calculate the CV score for different values of `max_depth` inside your classifier.

### Problem 3 Solution:

In [77]:
values = [2,5,10,50,100]

# Define base classifier again

for depth in values:
    
    #Define base classifier
    DT_clf = DecisionTreeClassifier(max_depth=depth)
    print(cross_val_score(DT_clf, X_train, y_train, cv=5))

[0.87061519 0.87572713 0.87025121 0.87483473 0.8704275 ]
[0.90816147 0.90860215 0.90013222 0.903658   0.90215954]
[0.92314472 0.92543628 0.92216836 0.92137506 0.92428383]
[0.90031729 0.90507668 0.90506831 0.89643015 0.90471573]
[0.90349022 0.90516482 0.90136624 0.89863376 0.90453944]


We get the best score for `max_depth=10`. Let's train and predict with that

In [78]:
# Define
DT_clf_best = DecisionTreeClassifier(max_depth=10)

# Fit
DT_clf_best.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=10)

In [79]:
y_train_pred = DT_clf_best.predict(X_train) 
y_test_pred = DT_clf_best.predict(X_test)

In [80]:
print('Balanced accuracy on the training set:',balanced_accuracy_score(y_train,y_train_pred))
print('Balanced accuracy on the test set:',balanced_accuracy_score(y_test,y_test_pred))

Balanced accuracy on the training set: 0.9190180705360326
Balanced accuracy on the test set: 0.8909501006423928


### Problem 4: Hyperparameter selection with the Cross-Validation grid search

For some algorithms you may need to optimize for more than one hyperparameter. For example, you can set the `max_features` hyperaparameter that controls how many features are being considered at each split. 

Writing multiple nested for loops, as we did above, may not be optimal or elegant. `Scikit` can do a grid-search Cross-Validation under the hood and give you the optimal combination of hyperparameters that give the best performing algorithm.

Use the Grid Search CV here: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html to find the optimal combination of `max_depth` and `max_features` (note: this number cannot be larger than 23, the number of features we have).

Report the training and test set balanced accuracy for the optimal combination of parameters.

Notice: This may take some time to run.


### Problem 4 Solution:

In [86]:
from sklearn.model_selection import GridSearchCV


# Define basic classifier
DT_clf = DecisionTreeClassifier()

# Parameter grid
param_grid = {'max_depth': [2,5,10,50],
              'max_features': [5,10,15,20]}

# Define grid scores
grid = GridSearchCV(DT_clf, param_grid)

# Fit
grid.fit(X_train, y_train)
print(grid.best_params_)

{'max_depth': 10, 'max_features': 20}


In [89]:
DT_best = DecisionTreeClassifier(max_depth=10, max_features=20)
DT_best.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=10, max_features=20)

Make Predictions

In [90]:
y_train_pred = DT_best.predict(X_train) 
y_test_pred = DT_best.predict(X_test)

In [91]:
print('Balanced accuracy on the training set:',balanced_accuracy_score(y_train,y_train_pred))
print('Balanced accuracy on the test set:',balanced_accuracy_score(y_test,y_test_pred))

Balanced accuracy on the training set: 0.9200793060929103
Balanced accuracy on the test set: 0.8929483924558567
